In [62]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans

# Carregando e limpando os dados

In [28]:
pagamentos = pd.read_csv('pagamentos.csv')
pagamentos.info()
print(pagamentos.nunique())
pagamentos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 427873 entries, 0 to 427872
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   data_publicacao          427873 non-null  object
 1   fase                     427873 non-null  object
 2   credor                   427873 non-null  object
 3   valor                    427873 non-null  object
 4   numero                   427873 non-null  object
 5   cpf_ou_cnpj              427873 non-null  object
 6   data_pagamento           427873 non-null  object
 7   numero_processo          253290 non-null  object
 8   bem_ou_servico_prestado  420978 non-null  object
 9   natureza                 422333 non-null  object
 10  acao                     407378 non-null  object
 11  funcao                   407414 non-null  object
 12  subfuncao                407405 non-null  object
 13  processo_licitatorio     407396 non-null  object
 14  fonte_recurso       

,data_publicacao,fase,credor,valor,numero,cpf_ou_cnpj,data_pagamento,numero_processo,bem_ou_servico_prestado,natureza,acao,funcao,subfuncao,processo_licitatorio,fonte_recurso
0,01/01/2010,EMPENHO,METALPIRES INDUSTRIA E COM LT,"R$ 203.690,00",10000348,06.965.538/0001-00,01/01/2010,157/2009,REF AQUISICAO DE MOBILIARIOS E EQUIPAMENTOS PA...,449052090000 - Out.Equip. e Mat. Permanente,2040 - Equipar e Reequipar Unidades Escolares ...,12 - EDUCACAO,361 - ENSINO FUNDAMENTAL,PREGAO,0001 - REC.IMP.TRANSF.EDUCACAO 25%
1,01/01/2010,EMPENHO,INST.NAC.DE SEGURIDADE SOCIAL,"R$ 1.500.000,00",10000195,01.404.357/0001-51,01/01/2010,NaN,REFERENTE A DESPESA COM PARCE- LAMENTO DA DI...,469071030000 - INSS/PARCELAMENTO,2145 - Administracao de encagos especiais,28 - ENCARGOS ESPECIAIS,843 - SERVICO DA DIVIDA INTERNA,ISENTO,0000 - RECURSOS ORDINARIOS
2,01/01/2010,EMPENHO,GASAUTO LOCAC?O DE VEICULOS L,"R$ 18.816,00",113/2009,00.426.288/0001-58,01/01/2010,039/2009,locac?o de 04 motocicletas 150cc com com parti...,339039780000 - locacao de veiculos,2141 - Seguranca no transito,26 - TRANSPORTE,452 - SERVICOS URBANOS,CONVITE,0050 - REC.PROPRIAS ENT.ADM.
3,02/01/2010,EMPENHO,SALUSTIANO JOSE ALVES DIAS,"R$ 4.800,00",10000024,407.981.865-34,02/01/2010,230/2007,"TERMO DE RENOVACAO DO ADITIVO 5025271/2009, R...",339039780000 - locacao de veiculos,2097 - Pavimentacao e infra-estrutura,15 - URBANISMO,451 - INFRA-ESTRUTURA URBANA,TOMADA DE PRECO,0000 - RECURSOS ORDINARIOS
4,02/01/2010,EMPENHO,VALDEANE DOS SANTOS DIAS,"R$ 3.600,00",10000494,006.534.185-66,02/01/2010,NaN,REF. ALUGUEL DO IMOVEL ONDE FUNCIONA O NUCL...,339039100000 - Locacao de imoveis,2084 - Programa Errad.Trabalho Infantil PETI,08 - ASSISTENCIA SOCIAL,243 - ASSISTENCIA A CRIANCA E AO ADO,ISENTO,0000 - RECURSOS ORDINARIOS


In [56]:
# Selecionando o ano desejado
pagamentos['data_pagamento'] = pd.to_datetime(pagamentos['data_pagamento'], dayfirst=True)
pagamentos = pagamentos[(pagamentos['data_pagamento'] > '2017-12-31') & (pagamentos['data_pagamento'] < '2019-01-01')]
min_data = pagamentos['data_pagamento'].min()
max_data = pagamentos['data_pagamento'].max()
print(f'Menor data: {min_data}, maior data: {max_data}')
pagamentos.head()

Menor data: 2018-01-01 00:00:00, maior data: 2018-12-27 00:00:00


,data_publicacao,fase,credor,valor,numero,cpf_ou_cnpj,data_pagamento,numero_processo,bem_ou_servico_prestado,natureza,acao,funcao,subfuncao,processo_licitatorio,fonte_recurso
341136,01/01/2018,EMPENHO,GREEN CARD S/A REFEIÇÕES COMERCIO E SERVIÇOS,"R$ 250.000,00",18000306,92.559.830/0001-71,2018-01-01,051-20171111,CONTRATAÇÃO DE EMPRESA PARA PRESTAÇÃO DE SERVI...,339039990000000000 - Outros Serv.Terceiros - P...,idica 2056 - Atenca...,10 - SAUDE,302 - ASSISTENCIA HOSPITALAR E AMBUL,PREGAO,0014 - TRANSF.REC. SUS
341137,03/01/2018,EMPENHO,COOPERSADE COOP.S.P.ESP.SAUDE,"R$ 150.000,00",18000026,03.817.692/0001-53,2018-01-03,347/2015,REF. A CONTRATAÇÃO DE EMPRESA ESPECIALIZADA NA...,339039990000000000 - Outros Serv.Terceiros - P...,idica 2194 - Serv.d...,08 - ASSISTENCIA SOCIAL,244 - ASSISTENCIA COMUNITARIA,PREGAO,0029 - TRANSF. REC. - FNAS
341138,03/01/2018,EMPENHO,COOPERSADE COOP.S.P.ESP.SAUDE,"R$ 214.000,00",18000027,03.817.692/0001-53,2018-01-03,347/2015,REF. A CONTRATAÇÃO DE EMPRESA ESPECIALIZADA NA...,339034010000000000 - Out.Serv.de Tercerizaçao-...,ídica 2239 - Serviç...,08 - ASSISTENCIA SOCIAL,244 - ASSISTENCIA COMUNITARIA,PREGAO,0000 - RECURSOS ORDINARIOS
341139,03/01/2018,EMPENHO,COOPERSADE COOP.S.P.ESP.SAUDE,"R$ 140.000,00",18000028,03.817.692/0001-53,2018-01-03,347/2015,REF. A CONTRATAÇÃO DE EMPRESA ESPECIALIZADA NA...,339039990000000000 - Outros Serv.Terceiros - P...,idica 2239 - Serviç...,08 - ASSISTENCIA SOCIAL,244 - ASSISTENCIA COMUNITARIA,PREGAO,0000 - RECURSOS ORDINARIOS
341140,03/01/2018,EMPENHO,COOPERSADE COOP.S.P.ESP.SAUDE,"R$ 80.000,00",18000029,03.817.692/0001-53,2018-01-03,347/2015,REF. A CONTRATAÇÃO DE EMPRESA ESPECIALIZADA NA...,339034010000000000 - Out.Serv.de Tercerizaçao-...,ídica 2239 - Serviç...,08 - ASSISTENCIA SOCIAL,244 - ASSISTENCIA COMUNITARIA,PREGAO,0029 - TRANSF. REC. - FNAS


In [57]:
# Selecionando a fase e colunas uteis
pagamentos = pagamentos[pagamentos['fase'] == 'PAGAMENTO']
pagamentos.drop(['data_publicacao', 'fase', 'numero', 
                'cpf_ou_cnpj', 'data_pagamento', 'numero_processo',
                'credor',  'bem_ou_servico_prestado', 'acao',
                'subfuncao', 'processo_licitatorio', 'fonte_recurso'],
                inplace=True, axis=1)
print(pagamentos.info())
print(pagamentos.nunique())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25713 entries, 341325 to 401630
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   valor     25713 non-null  object
 1   natureza  22699 non-null  object
 2   funcao    22699 non-null  object
dtypes: object(3)
memory usage: 803.5+ KB
None
valor       11535
natureza       95
funcao         20
dtype: int64


In [58]:
num_entradas_repetidas = len(pagamentos)
pagamentos = pagamentos.drop_duplicates()
num_entradas_unicas = len(pagamentos)
print(f'Numero de entradas duplicadas: {num_entradas_repetidas - num_entradas_unicas}\nNumero de entradas unicas: {num_entradas_unicas}')

Numero de entradas duplicadas: 13612
Numero de entradas unicas: 12101


In [59]:
# Limpando float
def currency_to_float(value):
    """Converte de R$ 69.848,70 (str) para 69848.70 (float)."""
    try:
        cleaned_value = value.replace("R$", "").replace(".", "").replace(",", ".")
        return float(cleaned_value)
    except ValueError:
        return

pagamentos['valor'] = pagamentos['valor'].apply(currency_to_float)
pagamentos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12101 entries, 341325 to 401592
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   valor     12101 non-null  float64
 1   natureza  10637 non-null  object 
 2   funcao    10637 non-null  object 
dtypes: float64(1), object(2)
memory usage: 378.2+ KB


In [61]:
print(pagamentos.info())
print(pagamentos.nunique())
pagamentos.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12101 entries, 341325 to 401592
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   valor     12101 non-null  float64
 1   natureza  10637 non-null  object 
 2   funcao    10637 non-null  object 
dtypes: float64(1), object(2)
memory usage: 378.2+ KB
None
valor       11535
natureza       95
funcao         20
dtype: int64


,valor,natureza,funcao
341325,548855.35,329021000000 - Juros s/Divida por Contrato,28 - ENCARGOS ESPECIAIS
341326,6544.08,339039930000 - Serviços Bancários,04 - ADMINISTRACAO
341336,1066.00,339039990000 - Outros Serv.Terceiros - P. Juri...,04 - ADMINISTRACAO
341916,26752.76,469071070000 - EMBASA,28 - ENCARGOS ESPECIAIS
341917,1723.26,339039930000 - Serviços Bancários,08 - ASSISTENCIA SOCIAL


# Visualizando os pagamentos

A visualização a seguir se inspira no scatter plot do [Painel COFOG](https://painel-cofog.tesouro.gov.br/dashboard_cofog.Rmd#section-especifica%C3%A7%C3%A3o-das-despesas---a%C3%A7%C3%A3o-de-governo).

Para isso, primeiro são definidos os clusters de acordo com o valor.

Depois, os gastos são separados e plotados por natureza.

- - - -

## Clustering

Primeiro, vamos visualizar os dados pra ver se existe alguma organização, algum insight sobre número de clusters ou qual algoritmo de cluster utilizar


vou usar um algoritmo ingênuo de cluster: KMeans.

KMeans começa com inicialização aleatória dos centros, então é suceptível a mínimos locais. 

O que se faz é rodar o algoritmo várias vezes e avaliar qual foi a melhor versão e utilizá-la.